## Understanding Variational Auto-Encoders

A notebook that explores the basic ideas behind VAEs

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
%matplotlib inline

np.random.seed(0)

In [0]:
# First looking at autoencoders
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train / 255.
x_test = x_test / 255.
x_train = np.reshape(x_train, (x_train.shape[0], -1))
x_test = np.reshape(x_test, (x_test.shape[0], -1))
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


In [0]:
img1 = np.expand_dims(x_train[0],axis=0)
img2 = np.expand_dims(x_train[1], axis=0)
plt.imshow(img1.reshape(28,28))
plt.figure()
plt.imshow(img2.reshape(28,28))

training_data = tf.placeholder(tf.float32, [None, x_train.shape[1]])
labels = tf.placeholder(tf.float32, [None, x_train.shape[1]])

output = training_data + labels

with tf.Session() as sess:
    print(img1.shape, img2.shape)
    print(training_data.shape)
    ret = sess.run([output], feed_dict={training_data: img1, labels: img2})
    print(ret[0].shape)
    plt.figure()
    plt.imshow(ret[0].reshape(28,28))